# Gilles Main - Do not touch !
## 1 - libraries setup
import general and custom libraries


In [1]:
# General Libraries
import numpy as np
import cv2
import opencv_jupyter_ui as jcv2
from tdmclient import ClientAsync

# Custom libraries 
from src.Vision import vision as vs              # Gilles Regamey
from src.Global_Nav import helpers_global as gn  # Julien Droulet
from src.Local_Nav import local_navigation as ln # Tom Rathjen
from src.Local_Nav import psymap as pm           # Gilles Regamey
from src.Filtering import KalmanFilter as kf     # Aubin sabatier
from src.Motion_Control import thymio as th      # ?

### Setup global variables and Constants
Map is a rectangle 1000x700 [mm]


In [2]:
MAP_SHAPE_MM = (1000,700)
MAP_SHAPE_CELL = (50,35)

#testing Constants
REFRAME = True 
VISUALIZE = True

#camera var
cap = cv2.VideoCapture(0)

#robot var
client = ClientAsync()
node = await client.wait_for_node() 
handle = await node.lock()


## Code for the Thymio

In [3]:
#kalmann

## Code to run locally

In [4]:
if REFRAME:
    Tmap = vs.get_warp(cap,MAP_SHAPE_MM,20,10)

ret,frame = cap.read()
if ret:
    if REFRAME:
        frame = cv2.warpPerspective(frame,Tmap,MAP_SHAPE_MM)
    fmap = vs.get_grid_fixed_map(frame,MAP_SHAPE_CELL)
    obscont = vs.get_obstacles(frame)
    print("Searching for destination...")
    while True:
        ret,frame = cap.read()
        if ret:
            if REFRAME:
                frame = cv2.warpPerspective(frame,Tmap,MAP_SHAPE_MM)
            ret,destmm = vs.get_destination(frame)
            if ret:
                dest = gn.convert_to_idx([coord / 10.0 for coord in destmm],2)
                dest[1]= 35-dest[1]
                dest = tuple(dest)
                break
            else:
                cv2.imshow()
        else:
            print("No camera !")
            break
    print("Found destination Point at {} [mm] {} [cells]".format(destmm,dest))
    print("Searching for Robot...")
    while True:
        ret,frame = cap.read()
        if ret:
            if REFRAME:
                frame = cv2.warpPerspective(frame,Tmap,MAP_SHAPE_MM)
            hls_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HLS_FULL) 
            ret,robpos,orient,pxpcm = vs.get_Robot_position_orientation(hls_frame)
            if ret:
                print("Robot found at {} [mm], {} [rad]".format(robpos,orient))
                break
        else:
            print("No camera !")
            break

start = gn.convert_to_idx(robpos/10.0,2)
start[1]= 35-start[1]
start = tuple(start)
path = gn.global_final(fmap,start,dest, "8N", False)


Computing warp matrix
Finished Warp Matrix computation.
Searching for destination...
Found destination Point at [222 471] [mm] (11, 12) [cells]
Searching for Robot...
Robot found at [737 324] [mm], 1.844455647876987 [rad]


In [5]:

while True:
    ret,frame = cap.read()
    if ret:
        if REFRAME:
            frame = cv2.warpPerspective(frame,Tmap,MAP_SHAPE_MM)
        hls_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HLS_FULL) 

        ret,robpos,orient,pxpcm = vs.get_Robot_position_orientation(hls_frameq)

        if VISUALIZE:
            vizu = vs.visualizer(hls_frame)
            omap =vs.grid_fixedmap_visualizer(fmap.transpose(),MAP_SHAPE_MM)
            obsimg = cv2.merge([omap,omap,omap])
            vizu = cv2.bitwise_or(vizu,obsimg)
            vizu = pm.hallucinate_map([robpos[0],robpos[1],orient],obscont,vizu)
            vizu = vs.draw_obstacles_poly(vizu,obscont,(255,255,0),2)
            vizu = cv2.circle(vizu,dest,20,(50,25,100),4)
            vizu = cv2.addWeighted(vizu,0.5,frame,0.5,0)
            vizu = vs.paint_robot(vizu,(0,0,200),robpos,orient,pxpcm)
            jcv2.imshow("Map",vizu)
        

    else:
        print("camera failure.")
        break

## closing calls


In [ ]:
#To unlock the Robot
await node.unlock()
jcv2.destroyAllWindows()